In [9]:
import pandas as pd
from pathlib import Path

# 设置路径
remap_path = Path("../query_yelp/ground_truth_dataset")

# 读取五个重编码后的表
df_business = pd.read_json("../query_yelp/origin_dataset/business_query.json", lines=True)
df_checkin  = pd.read_json("../query_yelp/origin_dataset/checkin_query.json", lines=True)
df_review = pd.read_json(
    "../query_yelp/origin_dataset/review_query.json",
    lines=True,
    convert_dates=False
)

df_tip      = pd.read_json("../query_yelp/origin_dataset/tip_query.json", lines=True)
df_user     = pd.read_json("../query_yelp/origin_dataset/user_query.json", lines=True)


In [2]:
df_business

,business_id,name,review_count,is_open,attributes,hours,description
0,businessid_49,Steps to Learning Montessori Preschool,8,1,"{'BusinessAcceptsCreditCards': 'True', 'WiFi':...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...","Located at 6901 Phelps Rd in Goleta, CA, this ..."
1,businessid_47,Breeze Blow Dry Bar,81,0,"{'ByAppointmentOnly': 'False', 'BusinessAccept...","{'Monday': '7:0-18:0', 'Tuesday': '7:0-18:0', ...","Located at 9916 Clayton Rd in St. Louis, MO, t..."
2,businessid_88,Impact Guns,39,1,"{'BusinessParking': '{'garage': False, 'street...","{'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'...","Located at 11655 W Executive Dr in Boise, ID, ..."
3,businessid_41,Palms Primary Care,5,1,None,"{'Monday': '8:30-17:0', 'Tuesday': '8:30-17:0'...","Located at 1615 Pasadena Ave S, Ste 430 in Sai..."
4,businessid_33,J&Q Nails,28,1,"{'BusinessParking': '{'garage': False, 'street...","{'Monday': '9:30-19:0', 'Tuesday': '9:30-19:0'...","Located at 9655 E US Hwy 36, Unit H in Avon, I..."
...,...,...,...,...,...,...,...
95,businessid_23,Harbor Freight,8,1,None,"{'Monday': '8:0-19:0', 'Tuesday': '8:0-19:0', ...","Located at 5211 Hickory Hollow Pkwy, Ste 103 i..."
96,businessid_38,Philadelphia Hair Studio,22,0,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","{'Tuesday': '11:0-19:0', 'Wednesday': '11:0-19...","Located at 744 S 6th St in Philadelphia, PA, t..."
97,businessid_81,Fantastic Sams Cut & Color,6,1,"{'BusinessParking': '{'garage': False, 'street...","{'Monday': '8:0-21:0', 'Tuesday': '8:0-21:0', ...","Located at 36129 E Lake Rd in Palm Harbor, FL,..."
98,businessid_13,Avian Glen Winery,12,1,"{'RestaurantsPriceRange2': '2', 'RestaurantsTa...","{'Monday': '12:0-20:0', 'Tuesday': '12:0-20:0'...","Located at 3545 Almaville Rd in Smyrna, TN, th..."


In [3]:
print (df_business.dtypes)

business_id     object
name            object
review_count     int64
is_open          int64
attributes      object
hours           object
description     object
dtype: object


In [46]:
import pandas as pd

# 读取数据
df = pd.read_json("../query_yelp/origin_dataset/business_query.json", lines=True)

def check_description(row):
    desc = row.get("description", "")
    if not isinstance(desc, str):
        return ["missing_description"]

    missing = []

    # 地址类字段严格匹配（原文必须完整出现）
    for field in ["address", "city", "state"]:
        val = str(row.get(field, ""))
        if val and val not in desc:
            missing.append(field)

    # 类别字段逐个词项匹配
    categories = row.get("categories", "")
    if isinstance(categories, str):
        category_list = [cat.strip() for cat in categories.split(",") if cat.strip()]
        for cat in category_list:
            if cat not in desc:
                missing.append(f"category:{cat}")

    return missing

# 应用检查
df["missing_fields"] = df.apply(check_description, axis=1)
failed = df[df["missing_fields"].apply(lambda x: len(x) > 0)]

# 输出结果
if failed.empty:
    print("✅ All descriptions passed — all fields and categories are present.")
else:
    print(f"❌ {len(failed)} rows failed. Missing info:")
    for idx, row in failed.iterrows():
        print(f"- Row {idx}: missing fields: {row['missing_fields']}")



❌ 1 rows failed. Missing info:
- Row 17: missing fields: ['category:Nightlife', 'category:Bars', 'category:Restaurants', 'category:Sports Bars']


In [47]:
import pandas as pd
from pathlib import Path

# 路径设置
input_path = Path("../query_yelp/origin_dataset/business_query.json")
output_path = Path("../query_yelp/origin_dataset/business_final.json")

# 读取数据
df = pd.read_json(input_path, lines=True)

# 删除四个字段
df = df.drop(columns=["address", "city", "state", "categories"]).copy()

# 保存结果
df.to_json(output_path, orient="records", lines=True, force_ascii=False)
print(f"✅ 已删除 address/city/state/categories 字段，保存到: {output_path}")



✅ 已删除 address/city/state/categories 字段，保存到: ..\query_yelp\origin_dataset\business_final.json


In [ ]:
import pandas as pd
from pathlib import Path

import json
from tqdm import tqdm
from openai import AzureOpenAI

client = AzureOpenAI(

)

deployment_name = "gpt-4o-mini" 
input_path = Path("../query_yelp/origin_dataset/business_cleaned.json")
output_path = Path("../query_yelp/origin_dataset/business_query.json")

df = pd.read_json(input_path, lines=True)
def generate_description(address, city, state, categories):
    fields = {
        "address": address,
        "city": city,
        "state": state,
        "categories": categories
    }

    # 检查是否至少有一个字段非空
    if not any(fields.values()):
        return None

    # 构建 location 描述部分
    location_parts = []
    for key in ["address", "city", "state"]:
        if fields[key]:
            location_parts.append(f"'{fields[key]}'")

    location_text = ", ".join(location_parts)
    location_clause = f"located at {location_text}" if location_text else ""

    # 构建类别部分
    if categories:
        category_clause = (
                f"The sentence must include the exact list of categories as a phrase: '{categories}', "
                "with each category included verbatim and unchanged."
            )

    else:
        category_clause = ""

    # 构建完整 prompt
    prompt = (
            f"Write a single, natural-sounding English sentence that describes a business {location_clause}. "
            f"{category_clause} "
            "Do not mention the business name. Use a helpful, informative, but concise tone."
        )

    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant writing business location descriptions."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=150
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"❌ Error generating for {location_text}: {e}")
        return None

# ========== 主循环 ==========
descriptions = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    desc = generate_description(
        address=row.get("address", ""),
        city=row.get("city", ""),
        state=row.get("state", ""),
        categories=row.get("categories", "")
    )
    descriptions.append(desc)

# ========== 写入 ==========
df["description"] = descriptions
df.to_json(output_path, orient="records", lines=True, force_ascii=False)
print(f"✅ 完整描述生成完毕，已保存至：{output_path}")

100%|██████████| 100/100 [01:33<00:00,  1.07it/s]

✅ 完整描述生成完毕，已保存至：..\query_yelp\origin_dataset\business_query.json


In [12]:
# 删除指定列
df_business_cleaned = df_business.drop(columns=["postal_code", "latitude", "longitude", "stars"]).copy()

# 保存到 origin_dataset 文件夹
output_path = Path("../query_yelp/origin_dataset")
output_path.mkdir(parents=True, exist_ok=True)
df_business_cleaned.to_json(output_path / "yelp_business_cleaned.json", orient="records", lines=True, force_ascii=False)

print("✅ df_business 已清洗并保存至 origin_dataset/yelp_business_cleaned.json")


✅ df_business 已清洗并保存至 origin_dataset/yelp_business_cleaned.json


In [32]:
df_checkin

,business_id,date
0,businessid_2,"2011-03-18 21:32:32, 2011-07-03 19:19:32, 2011..."
1,businessid_5,"2014-01-02 16:36:55, 2017-02-23 20:43:38, 2017..."
2,businessid_6,"2010-05-16 05:52:45, 2010-07-14 17:56:05, 2010..."
3,businessid_7,"2019-12-07 06:10:53, 2020-01-01 23:40:41, 2020..."
4,businessid_8,"2012-12-07 18:35:12, 2012-12-08 22:03:39, 2012..."
...,...,...
85,businessid_94,"2011-08-04 17:42:21, 2011-08-15 03:46:17, 2012..."
86,businessid_95,"2011-04-20 23:34:07, 2011-05-05 02:29:31, 2011..."
87,businessid_96,"2010-05-08 01:48:04, 2010-05-16 02:13:28, 2010..."
88,businessid_97,"2012-03-12 14:26:10, 2013-12-09 22:30:34"


In [10]:
df_review

,review_id,user_id,business_ref,rating,useful,funny,cool,text,date
0,reviewid_135,userid_548,businessref_34,2,0,0,0,"Sure, it's cheap, but there isn't much to see....","August 01, 2016 at 03:44 AM"
1,reviewid_1067,userid_213,businessref_89,5,2,0,0,Very good service but a little pricey for the ...,"June 14, 2021 at 11:39 AM"
2,reviewid_871,userid_616,businessref_82,4,0,0,0,My friend and I enjoyed a fantastic meal at Mi...,"29 May 2013, 23:01"
3,reviewid_314,userid_1903,businessref_66,2,1,2,1,This location is not one of my favorites peopl...,"21 May 2016, 18:48"
4,reviewid_487,None,businessref_95,1,0,0,0,Terrible service. I was charged twice for onli...,"November 01, 2021 at 05:11 PM"
...,...,...,...,...,...,...,...,...,...
1995,reviewid_687,None,businessref_16,2,5,0,0,The Fox has gone downhill since I last visited...,2014-02-06 16:08:41
1996,reviewid_202,userid_428,businessref_16,4,0,0,0,Food was great as well as the service. Big scr...,2015-02-24 06:44:00
1997,reviewid_1362,userid_1684,businessref_85,5,0,0,0,Excellent customer service. Our order took 10 ...,2017-12-03 03:37:00
1998,reviewid_1352,userid_13,businessref_96,5,2,0,1,"SO GOOD! I already love this place, even if I ...","March 08, 2011 at 11:20 PM"


In [5]:
import pandas as pd
import numpy as np
from pathlib import Path

# 随机覆盖 date 字段为自然语言格式之一
def randomize_date_column(df, date_col):
    dt = pd.to_datetime(df[date_col])
    
    fmt1 = dt.dt.strftime("%Y-%m-%d %H:%M:%S")        # ISO 风格
    fmt2 = dt.dt.strftime("%B %d, %Y at %I:%M %p")    # 美式自然语言
    fmt3 = dt.dt.strftime("%d %b %Y, %H:%M")          # 紧凑自然语言
    
    all_choices = pd.DataFrame({"f1": fmt1, "f2": fmt2, "f3": fmt3})
    df[date_col] = all_choices.apply(lambda row: np.random.choice(row.values), axis=1)
    return df

# 路径设置
remap_path = Path("../query_yelp/origin_dataset")
output_path = Path("../query_yelp/origin_dataset")
df_review = pd.read_json(remap_path / "review_query_remapped.json", lines=True)

# 重命名 stars → rating，随机格式化 date 列
df_review_cleaned = df_review.rename(columns={"stars": "rating"}).copy()
df_review_cleaned = randomize_date_column(df_review_cleaned, "date")

# 保存为 review_query.json
output_path.mkdir(parents=True, exist_ok=True)
df_review_cleaned.to_json(output_path / "review_query.json", orient="records", lines=True, force_ascii=False)

print("✅ 已将 date 列覆盖为三种自然语言格式之一并保存为 review_query.json")



✅ 已将 date 列覆盖为三种自然语言格式之一并保存为 review_query.json


In [13]:
import pandas as pd
import numpy as np
from pathlib import Path

# 随机覆盖 date 字段为自然语言格式之一
def randomize_date_column(df, date_col):
    dt = pd.to_datetime(df[date_col])
    
    fmt1 = dt.dt.strftime("%Y-%m-%d %H:%M:%S")        # ISO 风格
    fmt2 = dt.dt.strftime("%B %d, %Y at %I:%M %p")    # 美式自然语言
    fmt3 = dt.dt.strftime("%d %b %Y, %H:%M")          # 紧凑自然语言
    
    all_choices = pd.DataFrame({"f1": fmt1, "f2": fmt2, "f3": fmt3})
    df[date_col] = all_choices.apply(lambda row: np.random.choice(row.values), axis=1)
    return df

# 路径设置
remap_path = Path("../query_yelp/origin_dataset")
output_path = Path("../query_yelp/origin_dataset")
df_review = pd.read_json(remap_path / "user_query.json", lines=True)

# 重命名 stars → rating，随机格式化 date 列
df_review_cleaned = df_review.rename(columns={"stars": "rating"}).copy()
df_review_cleaned = randomize_date_column(df_review_cleaned, "yelping_since")

# 保存为 review_query.json
output_path.mkdir(parents=True, exist_ok=True)
df_review_cleaned.to_json(output_path / "user_query.json", orient="records", lines=True, force_ascii=False)

print("✅ 已将 date 列覆盖为三种自然语言格式之一并保存为 review_query.json")


✅ 已将 date 列覆盖为三种自然语言格式之一并保存为 review_query.json


In [34]:
df_tip

,user_id,business_ref,text,date,compliment_count
0,None,businessref_85,Great customer service. Great job !!!,2016-04-28 19:31:24,0
1,userid_965,businessref_12,Great place and some of the friendliest people...,2013-12-04 02:46:01,0
2,userid_909,businessref_96,Update: Blue Plate Specials will only be one M...,2015-06-23 00:22:09,0
3,None,businessref_45,Great produce section with competitive prices ...,2013-02-22 02:17:41,0
4,userid_1621,businessref_47,I know people come here for the blowouts - whi...,2015-09-19 18:58:40,0
...,...,...,...,...,...
779,userid_117,businessref_46,Best. Food. Ever. Also get the Dirty Bastard.,2016-03-09 01:20:58,0
780,userid_979,businessref_46,Love some trashed wings with a side of popover...,2020-03-21 01:31:11,0
781,None,businessref_27,This place is trash... never again...,2021-07-18 04:27:55,0
782,userid_10,businessref_8,Sign up and get $20 off your first uber ride! ...,2014-02-22 04:34:13,0


In [35]:
df_user

,user_id,name,review_count,yelping_since,useful,funny,cool,elite
0,userid_286,Todd,376,2009-01-15 16:40:06,1373,723,639,"2010,2011,2012,2013,2014"
1,userid_1331,Patt,1028,2010-07-13 15:42:03,9050,3249,5929,"2011,2012,2013,2014,2015,2016,2017,2018,2019,2..."
2,userid_1880,Norma,57,2010-09-07 23:24:36,217,57,115,"2012,2013"
3,userid_271,Antony,49,2011-10-23 19:47:29,116,159,34,
4,userid_534,Mandy,754,2011-08-30 13:46:26,2925,775,988,"2011,2012,2013,2014,2015,2016,2017,2018,2019,2..."
...,...,...,...,...,...,...,...,...
1994,userid_1023,Viet,1,2021-01-23 21:24:23,1,0,0,
1995,userid_1270,Deb,2,2012-10-03 23:10:28,5,0,0,
1996,userid_744,Debbie,2,2016-04-28 18:10:10,1,0,0,
1997,userid_818,Erica,1,2012-02-03 18:30:27,0,0,0,


In [29]:
import pandas as pd
from pathlib import Path


# 从 business 表构造 business_map：原始 business_id → businessid_N
business_ids = df_business["business_id"].tolist()  # 例：['businessid_1', ..., 'businessid_100']
original_ids = df_business.index.tolist() if df_business.index.name == 'business_id' else df_business["business_id"].index.tolist()
business_map = {original_id: business_id for original_id, business_id in zip(df_business["business_id"], business_ids)}

# 构建 business_ref_map，保持编号一致，只换前缀
business_ref_map = {
    original_id: business_id.replace("businessid_", "businessref_")
    for original_id, business_id in business_map.items()
}

# 替换 review 和 tip 中的 business_id
df_review = df_review.rename(columns={"business_id": "business_ref"}).copy()
df_review["business_ref"] = df_review["business_ref"].map(business_ref_map)

df_tip = df_tip.rename(columns={"business_id": "business_ref"}).copy()
df_tip["business_ref"] = df_tip["business_ref"].map(business_ref_map)

# 保存为新文件
output_path.mkdir(parents=True, exist_ok=True)
df_review.to_json(output_path / "review_query_rename.json", orient="records", lines=True, force_ascii=False)
df_tip.to_json(output_path / "tip_query_rename.json", orient="records", lines=True, force_ascii=False)

print("✅ df_review 和 df_tip 已保存为 review_query.json 与 tip_query.json")

# === 映射检查：打印前10个 business_id ↔ business_ref 对照 ===
print("\n📋 映射检查：原始 business_id 对应 businessid_* 和 businessref_*")
for i, (orig, bid) in enumerate(business_map.items()):
    if i >= 10: break
    ref = business_ref_map.get(orig, "❌ not mapped")
    print(f"原始ID: {orig} → business_id: {bid} → business_ref: {ref}")


✅ df_review 和 df_tip 已保存为 review_query.json 与 tip_query.json

📋 映射检查：原始 business_id 对应 businessid_* 和 businessref_*
原始ID: businessid_49 → business_id: businessid_49 → business_ref: businessref_49
原始ID: businessid_47 → business_id: businessid_47 → business_ref: businessref_47
原始ID: businessid_88 → business_id: businessid_88 → business_ref: businessref_88
原始ID: businessid_41 → business_id: businessid_41 → business_ref: businessref_41
原始ID: businessid_33 → business_id: businessid_33 → business_ref: businessref_33
原始ID: businessid_74 → business_id: businessid_74 → business_ref: businessref_74
原始ID: businessid_92 → business_id: businessid_92 → business_ref: businessref_92
原始ID: businessid_64 → business_id: businessid_64 → business_ref: businessref_64
原始ID: businessid_52 → business_id: businessid_52 → business_ref: businessref_52
原始ID: businessid_29 → business_id: businessid_29 → business_ref: businessref_29
